# library import

In [1]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# data load

In [2]:
data = pd.read_csv('/data/stock/stock.KOSPI.KOSDAQ.19960701to20220331.adjusted.datefilled.indicator.nonan.csv') # 절대 경로

## mk_cd 함수

In [3]:
## 함수
def mk_cd(x):
    if type(x) == str:
        if x.isalpha():
            return x
        else:
            if len(x) == 6:
                return x
            else:
                return "0"*(6-len(x))+x
    elif type(x) == int:
        x = str(x)
        return "0"*(6-len(x))+x

In [4]:
## 적용
data['code'] = list(map(mk_cd, data['code']))

In [5]:
data.shape

(11405883, 90)

In [96]:
data.info() # next_change 열도 따로 추가 안해도 됨.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11405883 entries, 0 to 11405882
Data columns (total 91 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   date                      int64  
 1   code                      object 
 2   open                      int64  
 3   high                      int64  
 4   low                       int64  
 5   close                     int64  
 6   volume                    int64  
 7   tradingvalue              int64  
 8   change                    float64
 9   survival                  int64  
 10  NT20D                     int64  
 11  previous_close            float64
 12  Next_Change               float64
 13  rTR                       float64
 14  rTR_5Dmax                 float64
 15  MA5                       float64
 16  MA20                      float64
 17  MA60                      float64
 18  MA120                     float64
 19  MA240                     float64
 20  AO                    

## EOM, EOMsig 열 float type으로 형 변환

In [6]:
data['EOM'] = data['EOM'].astype(float)
data['EOMsig'] = data['EOMsig'].astype(float)

In [7]:
data.head()

,date,code,open,high,low,close,volume,tradingvalue,change,survival,...,MACDsig,MI,PSARdown,PSARup,STC,TRIX,VIneg,VIpos,CR,DLR
0,19960701,000010,36496,37676,36496,37588,14499,542221500,1.18,1,...,0.000,1.0000,-1.0,-1.0,0.0,298.590000,0.000000,0.000000,0.00000,314.33000
1,19960702,000010,37588,38462,37151,37938,30476,1153293600,0.93,2,...,5.584,2.0177,-1.0,-1.0,0.0,0.001819,0.017550,0.065720,0.93115,0.93115
2,19960703,000010,38025,38025,37151,37719,18375,694638000,-0.58,3,...,10.869,2.9867,37676.0,-1.0,0.0,0.003636,0.059632,0.092241,0.34852,-0.57726
3,19960704,000010,37719,37938,37151,37370,22539,849041000,-0.93,4,...,10.096,3.9127,37676.0,37151.0,0.0,0.003554,0.085825,0.114870,-0.57997,-0.92526
4,19960705,000010,37151,38025,37151,38025,15830,595899300,1.75,5,...,15.993,4.8244,37938.0,37151.0,0.0,0.005599,0.107770,0.138710,1.16260,1.75270


In [7]:
data['MA240'][:100] # ma NaN 처리 안해도 됨...

0     37588.0
1     37763.0
2     37748.0
3     37654.0
4     37728.0
       ...   
95    36849.0
96    36831.0
97    36812.0
98    36789.0
99    36775.0
Name: MA240, Length: 100, dtype: float64

# 전처리 과정
1. next_change 5% 여부 열 추가 <br>
(식: d+1의 종가/d-0의 종가 > 1.05, <br>
즉 next_change / change > 1.05) <br>
2. scaling [10w] - 모든 가격 관련 변수를 전날 종가(previous_close열이 있음)로 나누기 (나누고 1 빼기)
3. d9d0.txt [11w]
4. d9d0.csv [11w]
5. 필요한 열만 남기기 [11w]

# next_change 5%

In [8]:
data['Next_Change'] / data['change']

0           0.788136
1          -0.623656
2           1.603448
3          -1.881720
4           0.982857
              ...   
11405878   -0.101041
11405879   -0.561462
11405880    5.739645
11405881    0.000000
11405882         inf
Length: 11405883, dtype: float64

In [10]:
data['close'] / data['previous_close']

0           1.000000
1           1.009311
2           0.994227
3           0.990747
4           1.017527
              ...   
11405878    1.297872
11405879    0.969945
11405880    1.016901
11405881    1.096953
11405882    1.000000
Length: 11405883, dtype: float64

In [11]:
data['Next_Change_5'] = data['Next_Change'] / data['change'] > 1.05
# 이제 columns 수 91

# scaling
* /project/dsml2022/inhovation97/code_for_paper/scaling1.py
* 파일 1~4까지 전부 스케일링 하는 피쳐는 동일
* -VI : VIneg
* +VI : VIpos
* BHB : BBH
* BLB : BBL
* Ichimoku -> 'Ichimoku_a', 'Ichimoku_b', 'ichimoku_base_line', 'ichimoku_conversion_line'
* Parabolic SAR -> 'PSARdown', 'PSARup'
* 'EOM', EMV' -> 'EOM'
* 'SR' -> 'SRSID', 'SRSIK'
<br>

---
삭제된 지표 <br>
* SMA
* WMA
* EMA

---
추가한 지표 <br>
* ADXneg
* ADXpos
* BBM
* BBP
* BBW
* KCP
* KCW
* DCP
* DCW

## 가격관련지표

In [12]:
가격관련지표 = ['open', 'high', 'low', 'close',
                'MA5', 'MA20', 'MA60', 'MA120', 'MA240',
                'VMAP', 'BBH', 'BBL', 'BBM', 'BBP', 'BBW',
                'KCH', 'KCL', 'KCM', 'KCP', 'KCW',
                'DCH', 'DCL', 'DCM', 'DCP', 'DCW',
                'Ichimoku_a', 'Ichimoku_b', 'ichimoku_base_line', 'ichimoku_conversion_line',
                'PSARdown', 'PSARup', 'KAMA','MACD',
                'volume','tradingvalue',
                'MFI', 'ADI', 'OBV',
                'CMF', 'FI', 'EOM', 'VPT', 'NVI', 'ATR', 'UI',
                'ADX', 'ADXneg', 'ADXpos',
                'VIneg', 'VIpos', 'TRIX', 'MI', 'CCI', 'DPO', 'KST',
                'STC', 'RSI', 'SRSI', 'SRSID', 'SRSIK', 'TSI', 'UO',
                'WR', 'AO', 'ROC', 'PPO', 'PVO']

In [13]:
len(가격관련지표)

67

## data의 cols에서 가격관련지표를 뺀 것

In [14]:
cols = list(data.columns.values)
for i in 가격관련지표:
    if i in cols:
        del cols[cols.index(i)]
cols

['date',
 'code',
 'change',
 'survival',
 'NT20D',
 'previous_close',
 'Next_Change',
 'rTR',
 'rTR_5Dmax',
 'PPOhist',
 'PPOsig',
 'PVOhist',
 'PVOsig',
 'SO',
 'SOsig',
 'EOMsig',
 'AIdown',
 'AIup',
 'KSTsig',
 'MACDdiff',
 'MACDsig',
 'CR',
 'DLR',
 'Next_Change_5']

## scaling by 전 날 종가

In [99]:
df_all = pd.DataFrame()
for i in tqdm(data['code'].unique()):
    ## 종목코드 별 df
    tmp_df = data.iloc[np.where(data['code'] == i)]
    
    
    tmp_가격관련지표 = tmp_df[가격관련지표].reset_index().drop('index', axis = 1)/tmp_df['previous_close'].reset_index().drop('index', axis = 1).values#/tmp_df['close'][:-1].reset_index().drop('index', axis = 1).values
    # 나눴을 때 제대로 된 값 안나오면 .values해서 나누기
    
    ## 나머지 cols df, reset_index해서 쓰기
    tmp_else = tmp_df[cols].reset_index().drop('index', axis = 1)
    
    tmp_all = pd.concat([tmp_가격관련지표, tmp_else], axis = 1)
    
    ## 아래로 이어붙이기, ignore_index하기
    df_all = pd.concat([df_all, tmp_all], ignore_index = True)


100%|█████████████████████████████████████| 4522/4522 [1:20:54<00:00,  1.07s/it]


# test (문제 발생 열 찾기)
* data.info() 한 cell 바로 아래에서 형 변환 함.

In [52]:
for i in range(67):
    tmp_df[가격관련지표].iloc[:,i].name, sum(tmp_df[가격관련지표].iloc[:,i])#.reset_index().drop('index', axis = 1)
# 'NaN' in tmp_df[가격관련지표].reset_index().drop('index', axis = 1) #/#
# 'NaN' in tmp_df['previous_close'].reset_index().drop('index', axis = 1).values

('open', 195525300)

('high', 201818353)

('low', 189831019)

('close', 194979668)

('MA5', 195479270.60000017)

...

('ADI', -195960922166.07834)

('OBV', 368304034733.0)

('CMF', -330.0588220226015)

('FI', 109560122195.5)

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [56]:
tmp_df[['EOM']].info() # 일단 nan 값 때문에 뜨는 에러는 아닌데

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6384 entries, 589280 to 595663
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   EOM     6384 non-null   object
dtypes: object(1)
memory usage: 357.8+ KB


In [60]:
tmp_df[['EOM']].iloc[1]

EOM    5157758483290.0
Name: 589281, dtype: object

In [63]:
sum_tmp = 0
for i in range(len(tmp_df[['EOM']])):
    sum_tmp += tmp_df[['EOM']].iloc[i]
#     if type(tmp_df[['EOM']].iloc[i]) == str:
#         print(tmp_df[['EOM']].iloc[i])

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [90]:
# 544, 545번째 eom이 문제 (float이 문자형으로 들어있음. type바꾸는 전처리 필요.)
# astype 이용
i
tmp_df[['EOM']].iloc[i].values[0]+tmp_df[['EOM']].iloc[i+1].values[0]
tmp_df[['EOM']].iloc[i-2].values[0]
tmp_df[['EOM']].iloc[i-1].values[0]
tmp_df[['EOM']].iloc[i].values[0]
tmp_df[['EOM']].iloc[i+1].values[0]

544

'-663582526080-676962225832'

404963471502.0

-483688007736.0

'-663582526080'

'-676962225832'

In [100]:
df_all

,open,high,low,close,MA5,MA20,MA60,MA120,MA240,VMAP,...,SOsig,EOMsig,AIdown,AIup,KSTsig,MACDdiff,MACDsig,CR,DLR,Next_Change_5
0,0.970948,1.002341,0.970948,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.991088,...,92.542,0.000000e+00,4.0,4.0,3143.300,0.0000,0.000,0.00000,314.33000,False
1,1.000000,1.023252,0.988374,1.009311,1.004656,1.004656,1.004656,1.004656,1.004656,1.001862,...,82.945,3.099408e+09,4.0,8.0,3152.900,22.3360,5.584,0.93115,0.93115,False
2,1.002293,1.002293,0.979256,0.994227,0.994992,0.994992,0.994992,0.994992,0.994992,0.992409,...,76.032,1.030060e+09,4.0,8.0,3155.600,21.1380,10.869,0.34852,-0.57726,True
3,1.000000,1.005806,0.984941,0.990747,0.998277,0.998277,0.998277,0.998277,0.998277,0.997031,...,60.003,6.360769e+08,16.0,8.0,3154.300,-3.0922,10.096,-0.57997,-0.92526,False
4,0.994140,1.017527,0.994140,1.017527,1.009580,1.009580,1.009580,1.009580,1.009580,1.006877,...,61.478,5.371003e+08,16.0,20.0,3155.200,23.5910,15.993,1.16260,1.75270,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11405878,0.992908,1.297872,0.985816,1.297872,1.070922,1.013121,1.175532,1.452411,1.815603,1.075390,...,80.342,5.724226e+07,68.0,100.0,-133.830,444.0000,-580.760,-44.20700,29.78700,False
11405879,1.060109,1.150273,0.942623,0.969945,0.859563,0.788525,0.903224,1.113661,1.396831,0.929344,...,74.620,8.594154e+07,64.0,96.0,-121.860,552.9900,-442.510,-45.88400,-3.00550,False
11405880,0.994366,1.022535,0.983099,1.016901,0.930141,0.824507,0.928901,1.143042,1.438310,0.964732,...,74.390,7.942500e+07,60.0,92.0,-106.320,615.4900,-288.640,-44.97000,1.69010,True
11405881,0.991690,1.185596,0.969529,1.096953,0.975069,0.826593,0.912687,1.119889,1.412853,1.014903,...,68.152,1.000615e+08,56.0,100.0,-86.160,736.7100,-104.460,-39.63400,9.69530,False


In [101]:
df_all.to_csv('new_df_scaled_yet_minus_1.csv', index = False)

# minus 1

In [15]:
df = pd.read_csv('new_df_scaled_yet_minus_1.csv')

## 오래걸려서 pandas로 안하기로.

In [104]:
df_all = pd.DataFrame()
for i in tqdm(data['code'].unique()):
    ## 종목코드 별 df
    tmp_df = df.iloc[np.where(data['code'] == i)]
    
    
    tmp_가격관련지표 = tmp_df[가격관련지표].reset_index().drop('index', axis = 1) - 1
    # 나눴을 때 제대로 된 값 안나오면 .values해서 나누기

    ## 나머지 cols df, reset_index해서 쓰기
    tmp_else = tmp_df[cols].reset_index().drop('index', axis = 1)
    
    tmp_all = pd.concat([tmp_가격관련지표, tmp_else], axis = 1)
    
    ## 아래로 이어붙이기, ignore_index하기
    df_all = pd.concat([df_all, tmp_all], ignore_index = True)

  0%|                                                  | 0/4522 [00:00<?, ?it/s]

,open,high,low,close,MA5,MA20,MA60,MA120,MA240,VMAP,...,SRSI,SRSID,SRSIK,TSI,UO,WR,AO,ROC,PPO,PVO
0,-0.029052,0.002341,-0.029052,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.008912,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000198,-1.000000,-1.000000,-1.000000,-1.000000
1,0.000000,0.023252,-0.011626,0.009311,0.004656,0.004656,0.004656,0.004656,0.004656,0.001862,...,-1.000000,-1.000000,-1.000000,-0.997340,-0.999159,-1.000709,-1.000000,-1.000000,-0.999998,-0.999784
2,0.002293,0.002293,-0.020744,-0.005773,-0.005008,-0.005008,-0.005008,-0.005008,-0.005008,-0.007591,...,-1.000000,-1.000000,-1.000000,-0.997401,-0.998939,-1.000996,-1.000000,-1.000000,-0.999998,-0.999785
3,0.000000,0.005806,-0.015059,-0.009253,-0.001723,-0.001723,-0.001723,-0.001723,-0.001723,-0.002969,...,-1.000000,-1.000000,-1.000000,-0.997473,-0.998995,-1.001473,-1.000000,-1.000000,-1.000000,-0.999737
4,-0.005860,0.017527,-0.005860,0.017527,0.009580,0.009580,0.009580,0.009580,0.009580,0.006877,...,-1.000000,-1.000000,-1.000000,-0.997516,-0.998697,-1.000595,-1.000000,-1.000000,-0.999997,-0.999782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,-0.027311,0.000000,-0.029412,-0.014706,-0.000840,0.026387,0.174286,0.409034,0.493319,0.026555,...,-0.999707,-0.999633,-0.999652,-1.009615,-0.982447,-1.024716,-1.132185,-1.001636,-1.002198,-1.029750
2055,-0.017058,-0.014925,-0.100213,-0.095949,-0.004691,0.026141,0.185885,0.425160,0.511557,0.029254,...,-0.999928,-0.999677,-0.999764,-1.010564,-0.985160,-1.041664,-1.128239,-1.004975,-1.002493,-1.029185
2056,-0.007075,0.037736,-0.009434,-0.004717,0.076415,0.119953,0.306274,0.571226,0.667311,0.124198,...,-0.999930,-0.999730,-0.999840,-1.012424,-0.981479,-1.046110,-1.149057,-1.005613,-1.002981,-1.031280
2057,0.042654,0.042654,-0.004739,0.042654,0.057346,0.117204,0.309147,0.572559,0.670711,0.123649,...,-0.999749,-0.999815,-0.999866,-1.012375,-0.981554,-1.036743,-1.161820,-1.006344,-1.002998,-1.031525


  0%|                                                  | 0/4522 [00:00<?, ?it/s]


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11405883 entries, 0 to 11405882
Data columns (total 91 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   open                      float64
 1   high                      float64
 2   low                       float64
 3   close                     float64
 4   MA5                       float64
 5   MA20                      float64
 6   MA60                      float64
 7   MA120                     float64
 8   MA240                     float64
 9   VMAP                      float64
 10  BBH                       float64
 11  BBL                       float64
 12  BBM                       float64
 13  BBP                       float64
 14  BBW                       float64
 15  KCH                       float64
 16  KCL                       float64
 17  KCM                       float64
 18  KCP                       float64
 19  KCW                       float64
 20  DCH                   

## open

In [17]:
f = open('new_df_scaled_yet_minus_1.csv', 'r')
lines = f.readlines()

In [18]:
len(lines)

11405884

In [19]:
all_lst = []
for i, line in tqdm(enumerate(lines)):
    line = line.strip().split(',') # 0~66번째까지가 가격관련지표
    if i == 0:
        code = line.index('code')
        date = line.index('date')
        surv = line.index('survival')
        NT20 = line.index('NT20D')
        nc5_ = line.index('Next_Change_5')
        pvcl = line.index('previous_close')
        l0 = line.copy()
        line.append('\n')
        all_lst.append(line)
        # code만 object
        # date, survival, NT20D 은 int
        # bool은 int로 바꾸기
    elif line.count('') < len(가격관련지표): # line[pvcl] != '':
        line[date] = int(line[date])
        line[surv] = int(line[surv])
        line[NT20] = int(line[NT20])
        line[nc5_] = int(bool(line[nc5_]))
        for fl in range(len(line)):
            if fl not in [code, date, surv, NT20, nc5_]:
                line[fl] = float(line[fl]) - 1 # minus 1까지 한 번에 처리하기
        line.append('\n')
        all_lst.append(line)
        
    elif line.count('') >= len(가격관련지표): # line[pvcl] == '': # 나누는 값이 nan이면 나눈 결과도 nan일테니까 -1 처리해줄 게 없음.
        line[date] = int(line[date])
        line[surv] = int(line[surv])
        line[NT20] = int(line[NT20])
        line[nc5_] = int(bool(line[nc5_]))
        for fl in range(len(가격관련지표)):
            if line[fl] == '':
                pass
        for fl in range(len(가격관련지표),len(line)):
            if fl not in [code, date, surv, NT20, nc5_]:
                if line[fl] =='':
                    pass
                else:
                    line[fl] = float(line[fl]) - 1
        
        
        line.append('\n')
        all_lst.append(line)
        
#         for enu, li in enumerate(line):
#             if enu not in [code, date, surv, NT20, nc5_]:
#                 li = float(li)
#             elif enu == code:
#                 pass # li[code] = li[code] # 그대로 cd
#             elif enu == date:
#                 li = int(li) # li[date] = int(li[date]) # dt
#             elif enu == surv:
#                 li = int(li) # li[surv] = int(li[surv]) # sv
#             elif enu == NT20:
#                 li = int(li) # li[NT20] = int(li[NT20]) # NT
#             elif enu == nc5_:
#                 li = int(bool(li)) # li[nc5_] = int(bool(li[nc5_])) # nc5
#         break

11405884it [20:21, 9340.62it/s] 


### test (버림)

In [164]:
len(line)
fl

91

91

In [160]:
# for lili in range(len(가격관련지표),len(line)+1):
#     lili
# len(가격관련지표)

In [146]:
data.loc[17013][가격관련지표]
data.loc[17013]['close']
data.loc[17013]['previous_close'] #아아... 이게 nan...
type(data.loc[17013]['previous_close'])

open     15100
high     15100
low      15100
close    15100
MA5        NaN
         ...  
WR         NaN
AO         NaN
ROC        NaN
PPO        NaN
PVO        NaN
Name: 17013, Length: 67, dtype: object

15100

nan

numpy.float64

In [140]:
df.loc[17013]

open               NaN
high               NaN
low                NaN
close              NaN
MA5                NaN
                 ...  
MACDdiff           NaN
MACDsig            NaN
CR                 NaN
DLR                NaN
Next_Change_5    False
Name: 17013, Length: 91, dtype: object

In [147]:
line.count('')

85

In [139]:
fl
i
line[fl]
line

0

17013

''

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 19961022,
 '000042',
 '0.0',
 1,
 1,
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 1]

In [166]:
all_lst[17013]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 19961022,
 '000042',
 -1.0,
 1,
 1,
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 1,
 '\n',
 '\n']

In [119]:
int(bool('True'))

1

In [20]:
len(all_lst)

11405884

## 저장 test

In [134]:
for i in tqdm(all_lst):
    for j in i:
        if j != '\n':
            f'{j},'
        else:
            f'{j}'
            break

  0%|                                                     | 0/2 [00:00<?, ?it/s]

'open,'

'high,'

'low,'

'close,'

...

'-1.0,'

'313.33,'

'1,'

'\n'

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 18.70it/s]


In [175]:
df.loc[9124303]['previous_close']
df.loc[9124303]['date']

2450.0

20210902

In [172]:
all_lst[9124303]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 20101101,
 '071955',
 -1.0,
 879,
 20,
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 1,
 '\n']

## 첫 try에서 실수로 '\n' 두 번 들어가서 하나 지움

In [167]:
for i,j in enumerate(all_lst):
    if j.count('\n') == 2:
        del j[-1]
        print(i)

17013
17015
17017
17019
17021
17023
17025
17027
17029
17031
17033
17035
17037
17039
17041
17043
17045
17047
17049
17051
17053
17055
17057
17059
17061
17063
17065
17067
17069
17071
17073
17075
17077
17079
17081
17083
17085
17087
17089
17091
17093
17095
17097
17099
17101
17103
17105
17107
17109
17111
17113
17115
17117
17119
17121
17123
85838
85840
85842
85844
85846
85848
85850
85852
85854
85856
85858
85860
85862
85864
85866
85868
85870
85872
85874
85876
85878
85880
85882
85884
85886
85888
85890
85892
85894
85896
85898
85900
85902
85904
85906
85908
85910
85912
85914
85916
85918
85920
85922
85924
85926
85928
85930
85932
85934
85936
85938
85940
85942
85944
85946
85948
85950
85952
85954
85956
85958
85960
85962
85964
85966
85968
85970
85972
85974
85976
85978
85980
85982
85984
85986
85988
85990
85992
85994
85996
85998
86000
86002
86004
86006
86008
86010
86012
86014
86016
86018
86020
86022
86024
86026
86028
86030
86032
86034
86036
86038
86040
86042
86044
86046
86048
86050
86052
86054
86056
8605

174148
174150
174152
174154
174156
174158
174160
174162
174164
174166
174168
174170
174172
174174
174176
174178
174180
174182
174184
174186
174188
174190
174192
174194
174196
174198
174200
174202
174204
174206
174208
174210
174212
174214
174216
174218
174220
174222
174224
174226
174228
174230
174232
174234
174236
174238
174240
174242
174244
174246
174248
174250
174252
174254
174256
174258
174260
174262
174264
174266
174268
174270
174272
174274
174276
174278
174280
174282
174284
174286
174288
174290
174292
174294
174296
174298
174300
174302
174304
174306
174308
174310
174312
174314
174316
174318
174320
174322
174324
174326
174328
174330
174332
174334
174336
174338
174340
174342
174344
174346
174348
174350
174352
174354
174356
174358
174360
174362
174364
174366
174368
174370
174372
174374
174376
174378
174380
174382
174384
174386
174388
174390
174392
174394
174396
174398
174400
174402
174404
174406
174408
174410
174412
174414
174416
174418
174420
174422
174424
174426
174428
174430
174432

383279
383281
383283
383285
383287
383289
383291
383293
383295
383297
383299
383301
383303
383305
383307
383309
383311
383313
383315
383317
383319
383321
383323
383325
383327
383329
383331
383333
383335
383337
383339
383341
383343
383345
383347
383349
383351
383353
383355
383357
383359
383361
383363
383365
383367
383369
383371
383373
383375
383377
383379
383381
383383
383385
383387
383389
383391
383393
383395
383397
383399
383401
383403
383405
383407
383409
383411
383413
383415
383417
383419
383421
383423
383425
383427
383429
383431
383433
383435
383437
383439
383441
383443
383445
383447
383449
383451
383453
383455
383457
383459
383461
383463
383465
383467
383469
383471
383473
383475
383477
383479
383481
383483
383485
383487
383489
383491
383493
383495
383497
383499
383501
383503
383505
383507
383509
383511
383513
383515
383517
383519
383521
383523
383525
383527
383529
383531
383533
383535
383537
383539
383541
383543
383545
383547
383549
383551
383553
383555
383557
383559
383561
383563

607586
607588
607590
607592
607594
607596
607598
607600
607602
607604
607606
607608
607610
607612
607614
607616
607618
607620
607622
607624
607626
607628
607630
607632
607634
607636
607638
607640
607642
607644
607646
607648
607650
607652
607654
607656
607658
607660
607662
607664
607666
607668
607670
607672
607674
607676
607678
607680
607682
607684
607686
607688
607690
607692
607694
607696
607698
607700
607702
607704
607706
607708
607710
607712
607714
607716
607718
607720
607722
607724
607726
607728
607730
607732
607734
607736
607738
607740
607742
607744
607746
607748
607750
607752
607754
607756
607758
607760
607762
607764
607766
607768
607770
607772
607774
607776
607778
607780
607782
607784
607786
607788
607790
607792
607794
607796
607798
607800
607802
607804
607806
607808
607810
607812
607814
607816
607818
607820
607822
607824
607826
607828
607830
607832
607834
607836
607838
607840
607842
607844
607846
607848
607850
607852
607854
607856
607858
607860
607862
607864
607866
607868
607870

798263
798265
798267
798269
798271
798273
798275
798277
798279
798281
798283
798285
798287
798289
798291
798293
798295
798297
798299
798301
798303
798305
798307
798309
798311
798313
798315
798317
798319
798321
798323
798325
798327
798329
798331
798333
798335
798337
798339
798341
798343
798345
798347
798349
798351
798353
798355
798357
798359
798361
798363
798365
798367
798369
798371
798373
798375
798377
798379
798381
798383
798385
798387
798389
798391
798393
798395
798397
798399
798401
798403
798405
798407
798409
798411
798413
798415
798417
798419
798421
798423
798425
798427
798429
798431
798433
798435
798437
798439
798441
798443
798445
798447
798449
798451
798453
798455
798457
798459
798461
798463
798465
798467
798469
798471
798473
798475
798477
798479
798481
798483
798485
798487
798489
798491
798493
798495
798497
798499
798501
798503
798505
798507
798509
798511
798513
798515
798517
798519
798521
798523
798525
798527
798529
798531
798533
798535
798537
798539
798541
798543
798545
798547

...

5849372
5849374
5849376
5849378
5849380
5849382
5849384
5849386
5849388
5849390
5849392
5849394
5849396
5849398
5849400
5849402
5849404
5849406
5849408
5849410
5849412
5849414
5849416
5849418
5849420
5849422
5849424
5849426
5849428
5849430
5849432
5849434
5849436
5849438
5849440
5849442
5849444
5849446
5849448
5849450
5849452
5849454
5849456
5849458
5849460
5849462
5849464
5849466
5849468
5849470
5849472
5849474
5849476
5849478
5849480
5849482
5849484
5849486
5849488
5849490
5849492
5849494
5849496
5849498
5849500
5849502
5849504
5849506
5849508
5849510
5849512
5849514
5849516
5849518
5849520
5849522
5849524
5849526
5849528
5849530
5849532
5849534
5849536
5849538
5849540
5849542
5849544
5849546
5849548
5849550
5849552
5849554
5849556
5849558
5849560
5849562
5849564
5849566
5849568
5849570
5849572
5849574
5849576
5849578
5849580
5849582
5849584
5849586
5849588
5849590
5849592
5849594
5849596
5849598
5849600
5849602
5849604
5849606
5849608
5849610
5849612
5849614
5849616
5849618
5849620


6126104
6126106
6126108
6126110
6126112
6126114
6126116
6126118
6126120
6126122
6126124
6126126
6126128
6126130
6126132
6126134
6126136
6126138
6126140
6126142
6126144
6126146
6126148
6126150
6126152
6126154
6126156
6126158
6126160
6126162
6126164
6126166
6126168
6126170
6126172
6126174
6126176
6126178
6126180
6126182
6126184
6126186
6126188
6126190
6126192
6126194
6126196
6126198
6126200
6126202
6126204
6126206
6126208
6126210
6126212
6126214
6126216
6126218
6126220
6126222
6126224
6126226
6126228
6126230
6126232
6126234
6126236
6126238
6126240
6126242
6126244
6126246
6126248
6126250
6126252
6126254
6126256
6126258
6126260
6126262
6126264
6126266
6126268
6126270
6126272
6126274
6126276
6126278
6126280
6126282
6126284
6126286
6126288
6126290
6126292
6126294
6126296
6126298
6126300
6126302
6126304
6126306
6126308
6126310
6126312
6126314
6126316
6126318
6126320
6126322
6126324
6126326
6126328
6126330
6126332
6126334
6126336
6126338
6126340
6126342
6126344
6126346
6126348
6126350
6126352


7988058
7988060
7988062
7988064
7988066
7988068
7988070
7988072
7988074
7988076
7988078
7988080
7988082
7988084
7988086
7988088
7988090
7988092
7988094
7988096
7988098
7988100
7988102
7988104
7988106
7988108
7988110
7988112
7988114
7988116
7988118
7988120
7988122
7988124
7988126
7988128
7988130
7988132
7988134
7988136
7988138
7988140
7988142
7988144
7988146
7988148
7988150
7988152
7988154
7988156
7988158
7988160
7988162
7988164
7988166
7988168
7988170
7988172
7988174
7988176
7988178
7988180
7988182
7988184
7988186
7988188
7988190
7988192
7988194
7988196
7988198
7988200
7988202
7988204
7988206
7988208
7988210
7988212
7988214
7988216
7988218
7988220
7988222
7988224
7988226
7988228
7988230
7988232
7988234
7988236
7988238
7988240
7988242
7988244
7988246
7988248
7988250
7988252
7988254
7988256
7988258
7988260
7988262
7988264
7988266
7988268
7988270
7988272
7988274
7988276
7988278
7988280
7988282
7988284
7988286
7988288
7988290
7988292
7988294
7988296
7988298
7988300
7988302
7988304
7988306


8780947
8780949
8780951
8780953
8780955
8780957
8780959
8780961
8780963
8780965
8780967
8780969
8780971
8780973
8780975
8780977
8780979
8780981
8780983
8780985
8780987
8780989
8780991
8780993
8780995
8780997
8780999
8781001
8781003
8781005
8781007
8781009
8781011
8781013
8781015
8781017
8781019
8781021
8781023
8781025
8781027
8781029
8781031
8781033
8914967
8914969
8914971
8914973
8914975
8914977
8914979
8914981
8914983
8914985
8914987
8914989
8914991
8914993
8914995
8914997
8914999
8915001
8915003
8915005
8915007
8915009
8915011
8915013
8915015
8915017
8915019
8915021
8915023
8915025
8915027
8915029
8915031
8915033
8915035
8915037
8915039
8915041
8915043
8915045
8915047
8915049
8915051
8915053
8915055
8915057
8915059
8915061
8915063
8915065
8915067
8915069
8915071
8915073
8915075
8915077
8915079
8915081
8915083
8915085
8915087
8915089
8915091
8915093
8915095
8915097
8915099
8915101
8915103
8915105
8915107
8915109
8915111
8915113
8915115
8915117
8915119
8915121
8915123
8915125
8915127


10758132
10758134
10758136
10758138
10758140
10758142
10758144
10758146
10758148
10758150
10758152
10758154
10758156
10758158
10758160
10758162
10758164
10758166
10758168
10758170
11210557
11210559
11210561
11210563
11210565
11210567
11210569
11210571
11210573
11210575
11210577
11210579
11210581
11210583
11210585
11210587
11210589
11210591
11210593
11210595
11210597
11210599
11210601
11210603
11210605
11210607
11210609
11305493
11305495
11305497
11305499
11305501
11305503
11305505
11305507
11305509
11305511
11305513
11305515
11305517
11305519
11305521
11305523
11381677
11381679
11381681
11381683
11381685
11381687
11381689
11381691
11381693
11381695
11381697
11381699
11381701
11381703
11381705
11381707
11381709
11381711
11381713
11381715
11381717
11381719
11381721
11418528
11418530
11418532
11418534
11418536
11418538
11418540
11418542
11418544
11418546
11418548
11418550
11418552
11418554
11418556
11418558
11418560
11418562
11418564
11418566
11418568
11418570
11418572
11418574
11418576
1

# scaled_fin 저장

In [21]:
with open(f'scaled_fin.csv', 'w', encoding = 'UTF-8') as ff:
    for i in tqdm(all_lst):
        for j in i:
            if j != '\n':
                _ = ff.write(f'{j},')
            else:
                _ = ff.write(f'{j}')
# data가 왜 늘었지

100%|████████████████████████████| 11405884/11405884 [17:51<00:00, 10640.42it/s]
